
# 🧪 Laboratório prático – Sockets em Python (Colab)

**Objetivo:** criar e executar **servidor** e **cliente** em **TCP** e **UDP**, comparar os comportamentos e registrar reflexões.

> Dica: Se algo travar, use **Ambiente de execução → Reiniciar ambiente de execução**.



## ✅ Preparação
1. Renomeie este arquivo para **`Laboratorio_Sockets_TCP_UDP_SeuNome.ipynb`**.
2. Confirme que o ambiente está **conectado** (ícone no canto superior direito).
3. Execute as células **na ordem** (Servidor → Cliente).



## 🗺️ Roteiro
1. **Servidor TCP** → aguarda conexão na porta 5000 e ecoa mensagens com `OK:`
2. **Cliente TCP** → conecta e troca 3 mensagens
3. **Servidor UDP** → recebe datagramas até receber o comando **`SAIR`**
4. **Cliente UDP** → envia 3 mensagens e, ao final, o comando **`SAIR`**
5. **Experimentos** → pequenas mudanças para observar efeitos
6. **Reflexões** → responda em células de texto


## 1️⃣ Servidor TCP – **execute antes do cliente**

In [ ]:

import socket
import threading

HOST = '127.0.0.1'  # localhost
PORT = 5000

def servidor_tcp():
    srv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen()
    print(f"🖥️ Servidor TCP aguardando conexão em {HOST}:{PORT}...")

    conn, addr = srv.accept()
    print(f"✅ Conectado por {addr}")

    try:
        while True:
            data = conn.recv(1024)
            if not data:
                break
            msg = data.decode()
            print(f"📨 Recebido: {msg}")
            conn.sendall(f"OK: {msg}".encode())
    finally:
        conn.close()
        srv.close()
        print("🔚 Servidor TCP encerrado.")

# Executa o servidor TCP em background
threading.Thread(target=servidor_tcp, daemon=True).start()


## 2️⃣ Cliente TCP – **execute depois do servidor**

In [ ]:

import socket
import time

HOST = '127.0.0.1'
PORT = 5000

time.sleep(1)  # aguarda o servidor iniciar
cli = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
cli.connect((HOST, PORT))
print("📡 Cliente TCP conectado!\n")

for msg in ["Olá, servidor!", "Tudo bem?", "Encerrando..."]:
    cli.sendall(msg.encode())
    resp = cli.recv(1024).decode()
    print("💬 Resposta:", resp)
    time.sleep(0.5)

cli.close()
print("🔚 Cliente TCP finalizado.")


## 3️⃣ Servidor UDP – encerra ao receber **`SAIR`**

In [ ]:

import socket
import threading

HOST = '127.0.0.1'
PORT = 6000

def servidor_udp():
    srv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    srv.bind((HOST, PORT))
    print(f"🖥️ Servidor UDP aguardando mensagens em {HOST}:{PORT}...")

    while True:
        data, addr = srv.recvfrom(1024)
        msg = data.decode().strip()
        if msg.upper() == "SAIR":
            print("🚪 Comando 'SAIR' recebido. Encerrando servidor UDP.")
            break
        print(f"📨 Recebido de {addr}: {msg}")
        srv.sendto(b"OK_UDP", addr)

    srv.close()
    print("🔚 Servidor UDP encerrado.")

# Executa o servidor UDP em background
threading.Thread(target=servidor_udp, daemon=True).start()


## 4️⃣ Cliente UDP – envia dados e depois **`SAIR`**

In [ ]:

import socket
import time

HOST, PORT = '127.0.0.1', 6000

cli = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
print("📡 Cliente UDP iniciado.\n")

for msg in ["Ping", "UDP é rápido!", "Até logo"]:
    cli.sendto(msg.encode(), (HOST, PORT))
    data, _ = cli.recvfrom(1024)
    print("💬 Resposta do servidor:", data.decode())
    time.sleep(0.4)

# Envia comando de encerramento para o servidor UDP
cli.sendto("SAIR".encode(), (HOST, PORT))
cli.close()
print("🔚 Cliente UDP finalizado (com comando SAIR).")



## 5️⃣ Experimentos – mexa e observe
1. **Troque a porta** do TCP (5000 → 5001) no servidor e no cliente. O que ocorre se o cliente usar a porta antiga?
2. **Mude as mensagens** do cliente TCP. O servidor imprime exatamente o que você enviou?
3. No TCP, **comente** a linha `conn.sendall(...)`. O cliente espera pela resposta? Algum erro aparece?
4. No UDP, **remova** o `srv.sendto(...)`. O cliente recebe algo?
5. **Aumente** os `time.sleep(...)` e observe o impacto na latência entre envios e respostas.



## 6️⃣ Reflexões – responda aqui (texto)
- **R1:** Em uma frase, qual a principal diferença entre TCP e UDP observada neste lab?  
  
- **R2:** O que acontece com o cliente se o servidor TCP parar de responder? E no UDP?  
  
- **R3:** Por que alguns jogos e streaming preferem UDP? Dê um exemplo do lab que apoie sua resposta.  
  
- **R4:** Como as portas desses servidores apareceriam no Security Group de uma EC2? Quais regras seriam necessárias?



## 7️⃣ Entrega
1. Clique em **Compartilhar** → **Qualquer pessoa com o link** → **Leitor**.
2. Envie o link pelo Moodle.
3. Garanta que seu notebook contenha:
   - Códigos **TCP** (servidor e cliente) e **UDP** (servidor com `SAIR` e cliente).
   - As **reflexões respondidas** em células de texto.
   - **Capturas de tela** das saídas de servidor/cliente (opcional, mas recomendado).
